In [10]:
import pandas as pd

# 列方向を省略しない
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [3]:
df_train = pd.read_csv("../data/train.csv")
df_test = pd.read_csv("../data/test.csv")

/tmp/ipykernel_11435/2602871660.py:1: DtypeWarning: Columns (39,40,42,55,58) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv("../data/train.csv")
/tmp/ipykernel_11435/2602871660.py:2: DtypeWarning: Columns (39,40,42,55,56,58,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv("../data/test.csv")


   target_ym  money_room building_id  building_status building_create_date  \
0     202007       38000     a000002                1  2014-06-27 21:18:41   
1     202107      103000     a000004                1  2017-02-05 20:43:26   
2     202101       50000     a000005                1  2014-06-27 20:32:50   
3     202007       37000     a000008                1  2014-06-27 23:06:27   
4     202101       33000     a000008                1  2014-06-27 23:06:27   

  building_modify_date  building_type building_name  building_name_ruby  \
0  2024-05-25 00:45:07              3   Plaisirりんくう                 NaN   
1  2024-08-17 12:12:51              1      向原パークハイツ                 NaN   
2  2023-09-17 23:53:57              3       エナージ美香保                 NaN   
3  2024-03-08 00:10:05              3       エルディム希望                 NaN   
4  2024-03-08 00:10:05              3       エルディム希望                 NaN   

  homes_building_name  ... drugstore_distance  bank_distance  \
0          プレジール

In [4]:
df_train.head()

,target_ym,money_room,building_id,building_status,building_create_date,building_modify_date,building_type,building_name,building_name_ruby,homes_building_name,...,drugstore_distance,bank_distance,shopping_street_distance,est_other_name,est_other_distance,statuses,parking_keiyaku,money_hoshou_company,free_rent_duration,free_rent_gen_timing
0,202007,38000,a000002,1,2014-06-27 21:18:41,2024-05-25 00:45:07,3,Plaisirりんくう,NaN,プレジール りんくう,...,NaN,NaN,NaN,NaN,NaN,110702/210202/220101/220201/220301/230101/2308...,NaN,NaN,NaN,NaN
1,202107,103000,a000004,1,2017-02-05 20:43:26,2024-08-17 12:12:51,1,向原パークハイツ,NaN,向原パークハイツ,...,435.0,NaN,NaN,小竹向原駅,320.0,110301/210101/210201/210301/220101/220201/2203...,NaN,初回賃料等の50％、更新時1年毎に1万円,NaN,NaN
2,202101,50000,a000005,1,2014-06-27 20:32:50,2023-09-17 23:53:57,3,エナージ美香保,NaN,エナージ美香保,...,NaN,NaN,NaN,NaN,NaN,110201/110901/121001/210101/210202/220101/2202...,NaN,？月額家賃の50％、毎年10，000円 ？月額家賃の100％,NaN,NaN
3,202007,37000,a000008,1,2014-06-27 23:06:27,2024-03-08 00:10:05,3,エルディム希望,NaN,エルディム希望,...,NaN,NaN,NaN,生鮮食品オンリーワン 山手店,1114.0,121002/210202/210301/220101/220201/220301/2205...,NaN,契約時保証委託料：22，000円 月額保証委託料：賃料総額の2．2％又は5．5％,NaN,NaN
4,202101,33000,a000008,1,2014-06-27 23:06:27,2024-03-08 00:10:05,3,エルディム希望,NaN,エルディム希望,...,NaN,NaN,NaN,福山市立泉小学校,469.0,110301/121002/220201/220301/220501/230101/2302...,NaN,利用可（相談）,NaN,NaN


In [43]:
train_columns = [
    'money_room',
    #'building_id',
    'building_status',
    'building_type',
    'unit_count',
    'building_structure',
    'total_floor_area',
    'building_area',
    'floor_count',
    'basement_floor_count',
    'year_built',
    'building_land_area',
    'land_area_all',
    'unit_area_min',
    'unit_area_max',
    'building_land_chimoku',
    'land_youto',
    'land_toshi',
    'land_chisei',
    'land_area_kind',
    'land_setback_flg',
    'land_setback',
    'land_kenpei',
    'land_youseki',
    'land_road_cond',
    #'land_seigen',
    'building_area_kind',
    'management_form',
    'management_association_flg',
    #'reform_exterior',
    #'reform_exterior_other',
    #'reform_exterior_date',
    #'reform_common_area',
    #'reform_common_area_date',
    #'building_tag_id',
    'room_floor',
    'balcony_area',
    'dwelling_unit_window_angle',
    'room_count',
    'unit_area',
    'floor_plan_code',
    #'reform_date',
    #'reform_place',
    #'reform_place_other',
    #'reform_wet_area',
    #'reform_wet_area_other',
    #'reform_wet_area_date',
    #'reform_interior',
    #'reform_interior_other',
    #'reform_interior_date',
    #'reform_etc',
    #'renovation_date',
    #'renovation_etc',
    'flg_open',
    'flg_own',
    'bukken_type',
    'flg_investment',
    'empty_number',
    #'empty_contents',
    #'addr1_1',
    #'addr1_2',
    #'eki_name1',
    #'bus_stop1',
    #'bus_time1',
    #'walk_distance1',
    #'rosen_name2',
    #'eki_name2',
    #'bus_stop2',
    #'bus_time2',
    #'walk_distance2',
    #'traffic_other',
    #'traffic_car',
    'snapshot_land_area',
    'snapshot_land_shidou',
    'land_shidou_a',
    'land_shidou_b',
    'land_mochibun_a',
    'land_mochibun_b',
    'house_area',
    'flg_new',
    'house_kanrinin',
    'room_kaisuu',
    'snapshot_window_angle',
    'madori_number_all',
    'madori_kind_all',
    'money_kyoueki',
    'money_kyoueki_tax',
    'money_rimawari_now',
    'money_shuuzen',
    'money_shuuzenkikin',
    #'money_sonota_str1',
    'money_sonota1',
    #'money_sonota_str2',
    'money_sonota2',
    #'money_sonota_str3',
    'money_sonota3',
    'parking_money',
    'parking_money_tax',
    'parking_kubun',
    'parking_distance',
    'parking_number',
    #'parking_memo',
    'genkyo_code',
    'usable_status',
    'usable_date',
    #'school_ele_name',
    'school_ele_distance',
    #'school_ele_code',
    #'school_jun_name',
    'school_jun_distance',
    #'school_jun_code',
    'convenience_distance',
    'super_distance',
    'hospital_distance',
    'park_distance',
    'drugstore_distance',
    'bank_distance',
    'shopping_street_distance',
    #'est_other_name',
    'est_other_distance',
    #'statuses',
    'parking_keiyaku',
    #'money_hoshou_company',
    'free_rent_duration',
    'free_rent_gen_timing']


test_columns = [
    #'building_id',
    'building_status',
    'building_type',
    'unit_count',
    'building_structure',
    'total_floor_area',
    'building_area',
    'floor_count',
    'basement_floor_count',
    'year_built',
    'building_land_area',
    'land_area_all',
    'unit_area_min',
    'unit_area_max',
    'building_land_chimoku',
    'land_youto',
    'land_toshi',
    'land_chisei',
    'land_area_kind',
    'land_setback_flg',
    'land_setback',
    'land_kenpei',
    'land_youseki',
    'land_road_cond',
    #'land_seigen',
    'building_area_kind',
    'management_form',
    'management_association_flg',
    #'reform_exterior',
    #'reform_exterior_other',
    #'reform_exterior_date',
    #'reform_common_area',
    #'reform_common_area_date',
    #'building_tag_id',
    'room_floor',
    'balcony_area',
    'dwelling_unit_window_angle',
    'room_count',
    'unit_area',
    'floor_plan_code',
    #'reform_date',
    #'reform_place',
    #'reform_place_other',
    #'reform_wet_area',
    #'reform_wet_area_other',
    #'reform_wet_area_date',
    #'reform_interior',
    #'reform_interior_other',
    #'reform_interior_date',
    #'reform_etc',
    #'renovation_date',
    #'renovation_etc',
    'flg_open',
    'flg_own',
    'bukken_type',
    'flg_investment',
    'empty_number',
    #'empty_contents',
    #'addr1_1',
    #'addr1_2',
    #'eki_name1',
    #'bus_stop1',
    #'bus_time1',
    #'walk_distance1',
    #'rosen_name2',
    #'eki_name2',
    #'bus_stop2',
    #'bus_time2',
    #'walk_distance2',
    #'traffic_other',
    #'traffic_car',
    'snapshot_land_area',
    'snapshot_land_shidou',
    'land_shidou_a',
    'land_shidou_b',
    'land_mochibun_a',
    'land_mochibun_b',
    'house_area',
    'flg_new',
    'house_kanrinin',
    'room_kaisuu',
    'snapshot_window_angle',
    'madori_number_all',
    'madori_kind_all',
    'money_kyoueki',
    'money_kyoueki_tax',
    'money_rimawari_now',
    'money_shuuzen',
    'money_shuuzenkikin',
    #'money_sonota_str1',
    'money_sonota1',
    #'money_sonota_str2',
    'money_sonota2',
    #'money_sonota_str3',
    'money_sonota3',
    'parking_money',
    'parking_money_tax',
    'parking_kubun',
    'parking_distance',
    'parking_number',
    #'parking_memo',
    'genkyo_code',
    'usable_status',
    'usable_date',
    #'school_ele_name',
    'school_ele_distance',
    #'school_ele_code',
    #'school_jun_name',
    'school_jun_distance',
    #'school_jun_code',
    'convenience_distance',
    'super_distance',
    'hospital_distance',
    'park_distance',
    'drugstore_distance',
    'bank_distance',
    'shopping_street_distance',
    #'est_other_name',
    'est_other_distance',
    #'statuses',
    'parking_keiyaku',
    #'money_hoshou_company',
    'free_rent_duration',
    'free_rent_gen_timing']

In [40]:
df_train = df_train[train_columns]

In [41]:
# lgihtGBMでroom_moneyを予測する
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# 特徴量とターゲットの設定
X = df_train.drop(columns=['money_room'])
y = df_train['money_room']

# データの分割
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# LightGBMのデータセット作成
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val)

# ハイパーパラメータの設定
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 31,
    'verbose': -1
}

# モデルの学習
model = lgb.train(params, train_data, valid_sets=[val_data], num_boost_round=1000)

# バリデーションデータでの予測
y_pred = model.predict(X_val)

# RMSEの計算
rmse = mean_squared_error(y_val, y_pred, squared=False)
print(f'Validation RMSE: {rmse}')

Validation RMSE: 20481.219419831094


/home/satoru/.cache/pypoetry/virtualenvs/signate-gis-data-challenge-1-RQApCED0-py3.10/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [44]:
df_test['money_room'] = model.predict(df_test[test_columns])

In [51]:
submission = df_test[['index', 'money_room']]
# indexを6桁でpadding
submission['index'] = submission['index'].astype(str).str.zfill(6)
submission.to_csv('../data/submission/submission.csv', index=False,header=False)

/tmp/ipykernel_11435/1220122536.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['index'] = submission['index'].astype(str).str.zfill(6)


In [52]:
!signate submit --competition-id=1512 '../data/submission/submission.csv'

You have successfully submitted your predictions.We will send you the submission result to your email address.
